## MNIST Digit Classification with ONNX and OpenCV

This notebook is mainly for me to make sure what I am doing in rust makes sense since as of authoring this I am still getting used to using the libraries and type system in rust. It demonstrates how to use an ONNX model trained on the MNIST dataset to classify digits from images. It utilizes OpenCV for image loading and preprocessing, and ONNX Runtime (`onnxruntime`) for running inference. I am also using np for the preprocessing. These libraries are pretty optimized in python, so it gives me a mild benchmark so that I know I am going in the right direction as I learn and optimize in rust.

### Code Explanation

The script performs the following steps:

1. **Loading the ONNX Model:**
   - Loads the pre-trained ONNX model (`mnist-12.onnx`) using `onnxruntime.InferenceSession`.

2. **Loading and Preprocessing the Image:**
   - Loads an image of the number `9` using OpenCV (`cv2`), converts it to grayscale, resizes it to 28x28 pixels, and normalizes pixel values to a range of [0, 1].

3. **Running Inference:**
   - Prepares the preprocessed image data in the expected format (1x1x28x28) and feeds it into the ONNX model for inference using `session.run`.

4. **Post-processing:**
   - Retrieves raw logits from the model output, applies softmax to obtain class probabilities, and determines the predicted digit by selecting the class with the highest probability.

5. **Output:**
   - Prints the predicted digit and the total execution time in milliseconds.

### Example Output



In [ ]:
import onnxruntime as ort
import cv2
import numpy as np
import time


def main():
    start_time = time.time()
    
    # Load the ONNX model
    model_path = 'mnist-12.onnx'
    session = ort.InferenceSession(model_path)

    # Load the image
    image_path = 'test_digit_data/9/2.jpg'
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Resize and normalize
    image = cv2.resize(image, (28, 28)).astype(np.float32) / 255.0

    # Add batch dimension and reshape as expected by the model
    input_data = image.reshape(1, 1, 28, 28)

    # Run inference
    raw_output = session.run(None, {'Input3': input_data})

    # Extract the raw output probabilities (before softmax)
    raw_probabilities = raw_output[0]

    # Apply softmax to get probabilities
    probabilities = np.exp(raw_probabilities) / np.sum(np.exp(raw_probabilities), axis=1)

    # Get the predicted digit (class with highest probability)
    predicted_digit = np.argmax(probabilities)

    # Print the result and inference time
    print(f'Predicted Digit: {predicted_digit}')

    end_time = time.time()
    total_time_ms = (end_time - start_time) * 1000
    print(f'Total Execution Time: {total_time_ms:.2f} milliseconds')


if __name__ == '__main__':
    main()


Predicted Digit: 9
Total Execution Time: 4.40 milliseconds
